In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.optimize import fsolve
from scipy.integrate import quad
#  The function "equilibrium_hatalgebra" contained equilibrium conditions, which allows us to calculate the equilibrium price with given set of taxes
#  The function "minuswelfare" contained equations to calculate Home's welfare.
    ## The only difference between the two minuswelfare function is the fisrt argument input: I do this because scipy.minimize cosiders the first argument as the unkown to be solved by default.
#  The function "minimization" is used to maximize Home's welfare to get optimal tax values and energy price.
#  The function "callback_pe" takes the tax values calculated by welfare maximization to back out corresponding energy price.
#  The function "callback" takes optimal taxes and energy price to recalculate all other values including world emissions, home emissions...

In [54]:
def comp_jbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df):
    
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    ## optimal values
    # jxbar_hat =   (1 - df['jxbar']) ** (-1) / (((1 - df['jxbar']) ** (-1) - 1) + (1 + (1 - alpha) * tb_mat[0]/pe) ** (-theta) * (1 + tb_mat[0]/pe) ** ((1 - alpha) * theta));
    jxbar_hat = pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) / ( df['jxbar'] * pe**(-alpha*theta) * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * (pe + (1-alpha) * tb_mat[0])**(-theta));
    j0_hat = (pe+tb_mat[0])**(-(1-alpha)*theta) / (df['jxbar'] * (pe+tb_mat[0])**(-(1-alpha)*theta) + (1-df['jxbar']) * pe**(-(1-alpha)*theta) );
    jmbar_hat = 1
    
    if tax_scenario['tax_sce']=='Unilateral':
        te=varphi;
        tb_mat[1]=1;
        
    if tax_scenario['tax_sce']=='purete':
        jxbar_hat = 1;   
        jmbar_hat = 1;
    
    if tax_scenario['tax_sce']=='puretc':
        te=tb_mat[0];
        jxbar_hat = 1;
        jmbar_hat = 1;
        tb_mat[1]=1;
    
    if tax_scenario['tax_sce']=='puretp':
        te=tb_mat[0];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        tb_mat[1]=1;

    if tax_scenario['tax_sce']=='EC_hybrid':
        te=varphi;
        jxbar_hat = 1;
        jmbar_hat = 1;
        tb_mat[1]=1;

    if tax_scenario['tax_sce']=='PC_hybrid':
        te=tb_mat[0];
        if tb_mat[1]<0:
            tb_mat[1]=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)));
    
    if tax_scenario['tax_sce']=='EP_hybrid':
        te=tb_mat[1];
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * (1 + tb_mat[0]/pe) ** (theta * (1 - alpha)));

    if tax_scenario['tax_sce']=='EPC_hybrid':
        te=varphi;
        if tb_mat[1]<0:
            tb_mat[1]=0
        jxbar_hat =  (1 - df['jxbar']) ** (-1) / ((1 - df['jxbar']) ** (-1) - 1 + (1 + (tb_mat[0] - tb_mat[2] * tb_mat[0])/pe) ** (theta * (1 - alpha)));
        jmbar_hat = (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1)) / (1 + ((1 - df['jmbar']) ** (-1) - 1) ** (-1) * ((pe + tb_mat[0])/(pe + tb_mat[1] * tb_mat[0])) ** (theta * (1 - alpha)))
        
    return (te, jxbar_hat, jmbar_hat, j0_hat, tb_mat)

## compute values for the incomplete beta functions
def imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar):
    def tempFunction(i, theta, sigmastar):
        return (i ** ((1 + theta) / theta - 1) * (1 - i) ** ((theta - sigmastar) / theta - 1)) 
    
    Bfunvec1_prime = quad(tempFunction,0,j0_prime, args=(theta, sigmastar))[0]
    Bfunvec2_prime = quad(tempFunction,0,jxbar_prime, args=(theta, sigmastar))[0]
    
    return (Bfunvec1_prime, Bfunvec2_prime)
    
## computes extraction values (home, foreign)
def compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df):
    if logit==1:
        epsilonS=beta*(1-gamma)/(1-gamma+gamma*petbte**beta)
        epsilonSstar=beta*(1-gamma)/(1-gamma+gamma*pe**beta)
        Qe_hat = (petbte)**beta/(1-gamma+gamma*(petbte)**beta)
        Qestar_hat = pe**beta/(1-gamma+gamma*pe**beta)
        
    ## compute hat values    
    Qe_hat = (petbte) ** epsilonS
    Qestar_hat = pe ** epsilonSstar
    
    ## compute final values
    Qe_prime = df['Qe'] * Qe_hat
    Qestar_prime = df['Qestar'] * Qestar_hat
    
    return (Qe_prime, Qestar_prime)


## computes consumption values (home, import, export, foreign)
def compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    Bfunvec1_prime, Bfunvec2_prime = imcomp_betas(j0_prime, jxbar_prime, theta, sigmastar)
    
    CeHH_hat = (pe + tb_mat[0]) ** (-epsilonD) * jmbar_hat ** (1 + (1 - sigma)/theta);
    CeHH_prime = df['CeHH'] * CeHH_hat;
       
    
    # CeFH_hat = (1 + (1 - sigmastar)/theta) * pe ** (-(1 - alpha) * sigmastar) * (pe + tb_mat[0]) ** (-alpha) * Bfunvec_prime/(df['jxbar'] ** (1 +1/theta)) * (1 - df['jxbar']) ** (sigmastar/theta);
    CeFH1_hat = (pe +tb_mat[0])**(-epsilonDstar) * j0_hat**(1 + (1 - sigmastar)/theta);
    CeFH2_hat = (1 + (1 - sigmastar)/theta) * ((1-df['jxbar'])/df['jxbar'])**(sigmastar/theta) * pe**(-epsilonDstar) * (1 + tb_mat[0]/pe)**(-alpha) * (Bfunvec2_prime - Bfunvec1_prime)/df['jxbar']**(1+(1-sigmastar)/theta);
    CeFH1_prime = df['CeFH'] * CeFH1_hat;
    CeFH2_prime = df['CeFH'] * CeFH2_hat;
    CeFH_hat = CeFH1_hat + CeFH2_hat;
    if tax_scenario['Base']==1:
        CeFH_hat = pe ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeFH_hat = (pe + tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) ** (-epsilonDstar) * jxbar_hat ** (1 + (1 - sigmastar)/theta);
    
    if np.isnan(CeFH_hat)==True:
        CeFH_hat=0
    CeFH_prime =df['CeFH'] * CeFH_hat;
    
    
    CeHF_hat = (pe + tb_mat[0]) ** (-epsilonD);
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        CeHF_hat = (pe) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        CeHF_hat = (pe + tb_mat[1] * tb_mat[0]) ** (-epsilonD) * ((1 - jmbar_prime)/(1 - df['jmbar'])) ** (1 + (1 - sigma)/theta);
    
    CeHF_prime = df['CeHF'] * CeHF_hat;
    
    
    CeFF_prime = df['CeFF'] * ((1 - jxbar_prime)/(1-df['jxbar'])) ** (1 + (1 - sigmastar)/theta) * pe ** (-epsilonDstar);
    
    return (CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat)



def comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList):
    
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    ##
    VgHH = df['CeHH']/(1 - alpha)
    VgFF = df['CeFF']/(1 - alpha)
    
    VgFH = df['CeFH'] /(1 - alpha)
    # VgFH_prime = VgFH * pe ** ((1 - sigmastar) * (1 - alpha)) * (1 - (1 - jxbar_prime) ** (1 + (1 - sigmastar)/theta))/ (df['jxbar'] * (1 - df['jxbar']) ** ( (1-sigmastar)/theta))
    VgFH1_hat = (pe + tb_mat[0]) * CeFH1_hat
    VgFH2_hat = pe**(1 - epsilonDstar) * ((1-j0_prime)**(1+(1-sigmastar)/theta) - (1-jxbar_prime)**(1+(1-sigmastar)/theta))/ (df['jxbar']  * (1 - df['jxbar'] )**( (1-sigmastar)/theta))
    VgFH1_prime = VgFH * VgFH1_hat
    VgFH2_prime = VgFH * VgFH2_hat
    VgFH_hat = VgFH1_hat + VgFH2_hat
    
    if tax_scenario['Base']==1:
        VgFH_hat = pe * CeFH_hat
        
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgFH_hat = (pe + tb_mat[0]) * CeFH_hat
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgFH_hat = (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_hat
        
    if np.isnan(VgFH_hat)==True:
            VgFH_hat=0
    VgFH_prime = VgFH * VgFH_hat

    VgHF = df['CeHF']/(1 - alpha)
    VgHF_hat = (pe + alpha * tb_mat[0]) * CeHF_hat
    
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        VgHF_hat = pe * CeHF_hat
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        VgHF_hat = (pe + alpha * tb_mat[1] * tb_mat[0]) * CeHF_hat
        
    VgHF_prime = VgHF * VgHF_hat
    
    return (VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF)

def comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime):
    Ve_prime=(pe+tb_mat[0]) * Ce_prime
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Ve_prime = (pe+tb_mat[0]) * CeHH_prime + pe * CeHF_prime
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Ve_prime = (pe+tb_mat[0]) * CeHH_prime + (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime
    
    Vestar_prime= (pe+tb_mat[0]) * CeFH_prime  + pe * CeFF_prime
    Vestar_prime= (pe+tb_mat[0]) * CeFH1_prime + pe * CeFH2_prime + pe * CeFF_prime
    
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vestar_prime = pe * Cestar_prime
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vestar_prime = (pe + tb_mat[0] - tb_mat[2]*tb_mat[0]) * CeFH_prime + pe * CeFF_prime
        
    return (Ve_prime, Vestar_prime)

def comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    Vg = df['Ce'] /(1-alpha)
    Vg_prime_hat = (pe + tb_mat[0]) * Ce_hat
    Vg_prime = Vg * Vg_prime_hat
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * pe
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vg_prime = CeHH_prime/(1-alpha) * (pe + tb_mat[0]) + CeHF_prime/(1-alpha) * (pe + tb_mat[1] * tb_mat[0]);   
    
    Vgstar = df['Cestar'] /(1-alpha)
    Vgstar_prime = VgFH_prime + CeFF_prime/(1-alpha)* pe
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid' :
        Vgstar_prime = Cestar_prime/(1-alpha)* pe
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Vgstar_prime = CeFF_prime/(1-alpha)* pe + CeFH_prime/(1-alpha)* (pe + tb_mat[0] - tb_mat[2]*tb_mat[0])
    
    return (Vg, Vg_prime, Vgstar, Vgstar_prime)

def comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario):
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    Lg = alpha/(1-alpha) * df['Ge']
    Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * Ge_prime
    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='EC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * pe * CeFH_prime
        
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lg_prime = alpha/(1-alpha) * (pe + tb_mat[0]) * CeHH_prime + alpha/(1-alpha) * (pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) * CeFH_prime    
    
    Lgstar = alpha/(1-alpha) * df['Gestar']
    Lgstar_prime = alpha/(1-alpha) * (pe+tb_mat[0]) * CeHF_prime +alpha/(1-alpha) * pe * CeFF_prime
    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        Lgstar_prime = alpha/(1-alpha) * pe * Gestar_prime
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        Lgstar_prime = alpha/(1-alpha) * (pe + tb_mat[1]*tb_mat[0]) * CeHF_prime + alpha/(1-alpha) * pe * CeFF_prime
        
    return (Lg, Lg_prime, Lgstar, Lgstar_prime)

def comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList):
    
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList
    
    if pe < 0:
        pe = 0.0001
    
    delta_Le = (epsilonS/(epsilonS + 1)) * df['Qe'] * (petbte**(epsilonS + 1) - 1);
    delta_Lestar = (epsilonSstar/(epsilonSstar + 1)) * df['Qestar'] * (pe**(epsilonSstar + 1) - 1);
    
    def Func(a, beta, gamma):
        return (((1-gamma)*a**beta)/(1-gamma+gamma*a**beta)**2)
    
    if logit==1:
        delta_Le = beta * df['Qe'] * quad(Func,1,petbte, args=(beta, gamma))[0];
        delta_Lestar = beta * df['Qestar'] * quad(Func,1,pe, args=(beta, gamma))[0];
       
    delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe + tb_mat[0]) + Vgstar * (1/theta) * math.log(df['jxbar']/j0_prime * (pe+tb_mat[0])**(-(1-alpha)*theta)) \
           - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
           + Vg * (alpha - 1) * math.log(pe +  tb_mat[0]) + Vgstar * (alpha - 1) * math.log(pe) \
           - varphi * (Qeworld_prime - df['Qeworld']);

    if tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EP_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               - varphi * (Qeworld_prime - df['Qeworld']);
    
    if tax_scenario['tax_sce']=='PC_hybrid' or tax_scenario['tax_sce']=='EPC_hybrid':
        delta_U = -delta_Le - delta_Lestar - (Lg_prime - Lg) - (Lgstar_prime - Lgstar) \
               +  Vg * ((alpha - 1) * math.log(pe + tb_mat[0]) + 1/theta * math.log(df['jmbar']/jmbar_prime)) \
               +  Vgstar * ((alpha - 1) * math.log(pe + tb_mat[0] - tb_mat[2] * tb_mat[0]) + 1/theta * math.log(df['jxbar']/jxbar_prime)) \
               - varphi * (Qeworld_prime - df['Qeworld']);
    return delta_Le, delta_Lestar, delta_U, pe

def comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df):
    
    leakage1 = -(Qestar_prime - df['Qestar'])/(Qeworld_prime - df['Qeworld']);
    leakage2 = -(Gestar_prime - df['Gestar'])/(Qeworld_prime - df['Qeworld']);
    leakage3 = -(Cestar_prime - df['Cestar'])/(Qeworld_prime - df['Qeworld']);   
    
    return (leakage1, leakage2, leakage3)
    
    

In [55]:
def equilibrium_hatalgebra(pe,*data):
    tb_mat, te, varphi, tax_scenario, ParaList, df = data
    alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

    ## compute extraction tax, and jbar's
    te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = comp_jbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

    jxbar_prime = jxbar_hat * df['jxbar']
    jmbar_prime = jmbar_hat * df['jmbar']
    j0_prime = j0_hat * df['jxbar']

    #if te is too large, HoGe stop producing
    petbte = pe + tb_mat[0] - te
    z = pe + tb_mat[0] >= te
    petbte = petbte * z
    
    ## compute home and foreign extractions
    Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

    ## compute consumptions around the world
    CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
    
    diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime)
    #print('diff='+str(diff))
    return diff

In [56]:
class taxrow:
    def __init__(self, df, tax_scenario, varphi, ParaList):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.ParaList = ParaList
        self.results = 0
        self.prop = 1
        self.prop2 = None
        
    def opt_tax(self):
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = self.ParaList
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi

        if tax_scenario['tax_sce']=='Baseline':
            self.tb=0
            self.prop=1
            self.te=0
            
        elif tax_scenario['tax_sce']=='PC_hybrid':
            res = minimize(self.minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te));
            tb_mat = res.x;
            self.tb=tb_mat[0];
            self.prop=tb_mat[1];   
            self.prop2=tb_mat[2];
            self.te=self.tb
    
        elif tax_scenario['tax_sce']=='EPC_hybrid':
            res = minimize(self.minuswelfare, [0, 1, 1], bounds=[(0,None),(0,1),(0, 1)],method='trust-constr',args=(te));
            tb_mat = res.x;
            self.tb=tb_mat[0];
            self.prop=tb_mat[1];   
            self.prop2=tb_mat[2];
            self.te=varphi
  
        elif tax_scenario['tax_sce']=='EP_hybrid':
            res = minimize(self.minuswelfare, [0, 1], bounds=[(0,None),(0,None)],method='L-BFGS-B', args=(te));
            tb_mat = res.x
            self.tb=tb_mat[0]
            self.te=tb_mat[1]  
            self.prop=te-self.tb
            
        elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp' or tax_scenario['tax_sce']=='EC_hybrid' :
            res = minimize(self.minuswelfare, [0, 1], method='nelder-mead', args=(te));
            tb_mat = res.x
            self.tb=tb_mat[0]
            self.prop=tb_mat[1]
            #print("tax returned is", tb_mat, "flag is", res.success, "tax scenario is", tax_scenario['tax_sce'])
            if tax_scenario['tax_sce']=='puretc' or tax_scenario['tax_sce']=='puretp':
                self.te=self.tb
                
            elif tax_scenario['tax_sce']=='Unilateral' or tax_scenario['tax_sce']=='EC_hybrid':
                self.te=self.varphi
                
        elif tax_scenario['tax_sce']=='purete':
            tb_mat = [0,1];
            res = minimize(self.minuswelfare_purete, 0, method='nelder-mead', args=(tb_mat));
            self.te = res.x[0]
            tb_mat = [0,1];
            self.tb=tb_mat[0];
            self.prop=tb_mat[1];  
            
    def minuswelfare(self, tb_mat, te):
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        ParaList = self.ParaList
        df = self.df
         #solve for equilibrium
        data = (tb_mat, te, varphi, tax_scenario, ParaList, df)
        pe = fsolve(self.equilibrium_hatalgebra,1,args=data, full_output = True)
        pe=pe[0][0]
        if pe-te<0 and tax_scenario['tax_sce']=='purete':
            pe=te
            
        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, ParaList, df)
            
        return obj

    
    def minuswelfare_purete(self, te, tb_mat):
         #solve for equilibrium
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        ParaList = self.ParaList
        df = self.df
        data = (tb_mat, te, varphi, tax_scenario, ParaList, df)
        pe = fsolve(self.equilibrium_hatalgebra,1,args=data)
        pe=pe[0]
        # print('pe='+str(pe))
        if pe-te<0 and tax_scenario['tax_sce']=='purete':
            pe=te
        
        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, ParaList, df)
        
        #minuswelfare.welfare = delta_U/Vg;  
        #ob=-minuswelfare.welfare;
        return obj
        
        
    def equilibrium_hatalgebra(self, pe,*data):
        tb_mat, te, varphi, tax_scenario, ParaList, df = data
        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit = ParaList

        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = comp_jbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar']
        jmbar_prime = jmbar_hat * df['jmbar']
        j0_prime = j0_hat * df['jxbar']

        #if te is too large, HoGe stop producing
        petbte = pe + tb_mat[0] - te
        z = pe + tb_mat[0] >= te
        petbte = petbte * z

        ## compute home and foreign extractions
        Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        ## compute consumptions around the world
        CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
        diff = Qe_prime + Qestar_prime - (CeHH_prime + CeFH_prime + CeHF_prime + CeFF_prime)
        #print('diff='+str(diff))
        return diff
    
    ## compute the objective value, currently the objective is to minimize negative welfare
    ## also saves optimal results in self.
    def comp_obj(self, pe, te, tb_mat, varphi, tax_scenario, Paralist, df):

        alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS, epsilonSstar, beta, gamma, logit = ParaList
        ## compute extraction tax, and jbar's
        te, jxbar_hat, jmbar_hat, j0_hat, tb_mat = comp_jbar(ParaList, pe, te, varphi, tb_mat, tax_scenario, df)

        jxbar_prime = jxbar_hat * df['jxbar'];
        jmbar_prime = jmbar_hat * df['jmbar'];
        j0_prime = j0_hat * df['jxbar'];

        #if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te;
        z = pe + tb_mat[0] >= te;
        petbte = petbte * z;

        Qe_prime, Qestar_prime = compute_qe(petbte, epsilonS, epsilonSstar, logit, beta, gamma, pe, df)

        CeHH_prime, CeFH1_prime, CeFH2_prime, CeFH_prime, CeHF_prime, CeFF_prime, CeFH_hat, CeFH1_hat, CeHF_hat = compute_ce(pe, tb_mat,j0_hat, j0_prime, jmbar_hat, jmbar_prime,jxbar_hat, jxbar_prime, ParaList, df, tax_scenario)
        
        VgHH, VgFF, VgFH1_prime, VgFH2_prime, VgFH_prime, VgHF_prime, VgFH, VgHF = comp_vg(pe, tb_mat, CeFH1_hat, j0_prime, jxbar_prime, CeFH_hat, CeHF_hat, df, tax_scenario, ParaList)

        Ge_prime = CeHH_prime + CeFH_prime;
        #Ge_hat = Ge_prime/df['Ge'];
        Gestar_prime = CeFF_prime + CeHF_prime;
        Ce_prime = CeHH_prime + CeHF_prime;
        Ce_hat = Ce_prime/df['Ce'];
        Cestar_prime = CeFF_prime + CeFH_prime;
        Qeworld_prime=Qe_prime+Qestar_prime;
        pai_g = VgFH - (pe + tb_mat[0]) * df['CeFH'] / (1 - alpha);
        subsidy_ratio = (tb_mat[0]/pe * (1 - alpha)) / (1 + tb_mat[0]/pe * (1 - alpha));

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, Ce_prime, tax_scenario, CeHH_prime, CeHF_prime, CeFH_prime, CeFH1_prime, CeFH2_prime, CeFF_prime, Cestar_prime)

        ## compute more Vg values
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, Ce_hat, Cestar_prime, CeHH_prime, CeHF_prime, CeFH_prime, CeFF_prime,VgFH_prime, ParaList, df, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, VgFH_prime, Ce_hat, Ge_prime, Gestar_prime, CeHH_prime, CeFH_prime, CeHF_prime, CeFF_prime, ParaList, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        delta_Le, delta_Lestar, delta_U, pe = comp_delta(Lg, Lg_prime, Lgstar, Lgstar_prime, Qeworld_prime, Vg, Vgstar, df, j0_prime, jmbar_prime, jxbar_prime, pe, petbte, tb_mat, tax_scenario, varphi, ParaList)
        
        chg_extraction=Qestar_prime-df['Qestar']
        chg_production=Gestar_prime-df['Gestar']
        chg_consumption=Cestar_prime-df['Cestar']
        chg_Qeworld=Qeworld_prime-df['Qeworld']
        
        welfare = delta_U/Vg*100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld']) )/Vg*100
        
        self.results = (pd.Series({'varphi': varphi, 'pe': pe, 'tb': 0, 'prop': 0, 'jxbar_prime': jxbar_prime, 'jmbar_prime': jmbar_prime, 'j0_prime': j0_prime, \
              'Qe_prime': Qe_prime, 'Qestar_prime': Qestar_prime, 'Qeworld_prime': Qeworld_prime, \
              'CeHH_prime': CeHH_prime,'CeFH_prime': CeFH_prime, 'CeHF_prime': CeHF_prime, 'CeFF_prime': CeFF_prime,\
              'Ge_prime': Ge_prime,'Ce_prime': Ce_prime, 'Gestar_prime': Gestar_prime,'Cestar_prime': Cestar_prime, \
              'VgFH_prime': VgFH_prime, 'VgHF_prime': VgHF_prime, \
              'VgFH1_prime': VgFH1_prime, 'VgFH2_prime': VgFH2_prime, \
              'CeFH1_prime': CeFH1_prime, 'CeFH2_prime': CeFH2_prime, \
              'Vg_prime': Vg_prime, 'Vgstar_prime': Vgstar_prime, \
              'Lg_prime': Lg_prime, 'Lgstar_prime': Lgstar_prime, \
              'Ve_prime': Ve_prime, 'Vestar_prime': Vestar_prime, \
              'delta_Le': delta_Le, 'delta_Lestar': delta_Lestar, \
              'leakage1': leakage1, 'leakage2': leakage2,'leakage3': leakage3, \
              'leakage3': leakage3, 'chg_extraction': chg_extraction, 'chg_production': chg_production, \
              'chg_consumption': chg_consumption,'chg_Qeworld':chg_Qeworld, 'pai_g': pai_g, \
              'subsidy_ratio': subsidy_ratio, 'welfare':welfare, 'welfare_noexternality':welfare_noexternality}))

        return -delta_U/Vg
    
    
    def retrive(self):
        #print(self.results)
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        ret['prop2'] = self.prop2
        return ret

In [8]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize, least_squares
from scipy.optimize import fsolve
from scipy.integrate import quad
import matplotlib.pyplot as plt

In [10]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [11]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5;  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5;  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [12]:
ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)

In [13]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['region_scenario','regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF']);
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF']);

## choose which regional scenario to run (runs all if not executed)
df=df.drop([2,3,4,5,6,7,8,9,10])  
df

,,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
region_scenario,regionbase,,,,,,,,,,,,,,,
1,US as Home,4.480045,27.795946,32.27599,4.598403,1.196111,0.421601,26.059874,5.794514,26.481474,5.020004,27.255985,32.27599,32.27599,0.015921,0.793579


In [14]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[2, 3, 4, 5, 6, 7, 8, 9])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [68]:
def temp(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    te = 0; #initial value of extraction tax for iteration
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    varphi = 2
    tax_df=df.apply(mintemp, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
    varphilist = np.arange (0,20,0.1) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = [2]
    output=[]
    for varphi in varphilist:    
        te = 0; #initial value of extraction tax for iteration
        tb_mat = [0, 1]  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)

        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
        
        output.append(tax_df)
        #print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    #print(tax_df)
    
    # appended_df = pd.merge(df, tax_df, on=['region_scenario','regionbase'])
    # output_df = appended_df.apply(callback, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
    # output_df = pd.merge(tax_df, output_df, on=['region_scenario','regionbase'])
    # output = [tax_df]
    # output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    # output.reset_index(level=0, inplace=True)
    # output = output.sort_values(by=['region_scenario','varphi'])
    #print(output)
    return output

In [69]:
def mintemp(df, tb_mat, te, varphi, tax_scenario, ParaList):
    tax_temp = taxrow(df, tax_scenario, varphi, ParaList)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [ ]:
temp_df = tax_scenario.apply(temp, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))
#temp_df.head()

C:\Users\Alex\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [63]:
temp_df.loc[2]

,,varphi,pe,tb,prop,jxbar_prime,jmbar_prime,j0_prime,Qe_prime,Qestar_prime,Qeworld_prime,...,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality,te,prop2
region_scenario,regionbase,,,,,,,,,,,,,,,,,,,,,
1,US as Home,0.9,1.002402,0.313793,-30.35437,0.016239,0.793579,0.013553,2.890217,27.829304,30.719521,...,0.033358,-0.358201,-0.164429,-1.556469,-0.888719,0.04485,1.782399,-1.843847,0.9,None


In [67]:
output_list=[]
for i in range(2,len(tax_scenario)+2):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

,tax_sce,region_scenario,varphi,pe,tb,prop,jxbar_prime,jmbar_prime,j0_prime,Qe_prime,...,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality,te,prop2
regionbase,,,,,,,,,,,,,,,,,,,,,
US as Home,Unilateral,1,0.0,1.000000,4.482269e-08,1.021871,0.015921,0.793579,0.015921,4.480045,...,-6.169471e-07,0.000001,0.000002,0.0,-1.254787e-09,6.725639e-09,0.000001,0.000001,0.0,None
US as Home,Unilateral,1,0.1,0.998753,3.473866e-02,-2.460976,0.015925,0.793579,0.015602,4.328501,...,-1.733880e-02,-0.006344,0.018898,-0.168882,-9.413360e-02,5.190227e-03,0.021883,-0.021835,0.1,None
US as Home,Unilateral,1,0.2,0.997896,6.941858e-02,-5.867249,0.015939,0.793579,0.015301,4.172253,...,-2.925719e-02,-0.020979,0.028409,-0.337048,-1.891992e-01,1.032698e-02,0.087375,-0.087125,0.2,None
US as Home,Unilateral,1,0.3,0.997416,1.040787e-01,-9.398399,0.015961,0.793579,0.015014,4.010815,...,-3.594001e-02,-0.043762,0.028753,-0.505169,-2.852676e-01,1.541104e-02,0.196379,-0.195934,0.3,None
US as Home,Unilateral,1,0.4,0.997304,1.387568e-01,-12.852244,0.015991,0.793579,0.014741,3.843608,...,-3.750018e-02,-0.074653,0.020046,-0.673936,-3.824213e-01,2.044315e-02,0.348972,-0.348863,0.4,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
US as Home,EPC_hybrid,1,0.5,0.998188,1.730305e-01,0.992084,0.015031,0.793464,0.014486,3.670409,...,-2.518922e-02,-0.102193,0.011073,-0.834824,-4.812404e-01,2.534273e-02,0.539603,-0.540933,0.5,0.410169
US as Home,EPC_hybrid,1,0.6,0.998928,2.077491e-01,0.992668,0.014880,0.793455,0.014239,3.489484,...,-1.490054e-02,-0.147465,-0.014240,-1.005461,-5.809022e-01,3.025206e-02,0.777868,-0.783806,0.6,0.417007
US as Home,EPC_hybrid,1,0.7,1.000045,2.426285e-01,0.992133,0.014736,0.793428,0.014002,3.300286,...,6.246925e-04,-0.201209,-0.049040,-1.179133,-6.820758e-01,3.511472e-02,1.0607,-1.075958,0.7,0.423937
